In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
# Estilos de Seaborn
sns.set()
sns.set_palette(palette='colorblind')
%matplotlib inline

Armado de set de "entrenamiento"

In [29]:
conversiones = pd.read_pickle("../data/conversiones.pkl")

In [30]:
conversiones.head()

,mes,anio,USER_ID,trimestre
0,1,2018,2483,1
1,1,2018,7088,1
2,1,2018,1787,1
3,1,2018,2085,1
4,1,2018,8270,1


In [64]:
#Me quedo con las columnas USER_ID y trimestre
df_aux=conversiones.loc[:,['USER_ID','trimestre']]

#Me quedo solo con las conversiones del trimestre que me interesa
cuatrimestre_de_interes=2
df_aux=df_aux[df_aux['trimestre'] == cuatrimestre_de_interes]

df_aux.head()

,USER_ID,trimestre
277,7545,2
278,5297,2
279,1493,2
280,8024,2
281,10551,2


In [65]:
#nueva columna indicando que esos users convirtieron
df_aux['convirtio']=True
df_aux.head()

,USER_ID,trimestre,convirtio
277,7545,2,True
278,5297,2,True
279,1493,2,True
280,8024,2,True
281,10551,2,True


In [66]:
df_aux.shape

(466, 3)

In [67]:
#Me deshago de los users repetidos (convirtieron mas de una vez por cuatrimestre)
df_aux.drop_duplicates(['USER_ID'], keep='first', inplace=True)
df_aux.shape

(374, 3)

In [69]:
df_aux.set_index('USER_ID',inplace=True)
df_aux=df_aux.loc[:,['convirtio']]
df_aux.head()

,convirtio
USER_ID,
7545,True
5297,True
1493,True
8024,True
10551,True
